In [2]:
import tensorflow as tf

from ocddetection.data import SENSORS
from ocddetection.models import bidirectional

In [3]:
model = bidirectional(150, len(SENSORS), 64, 0.6)

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block_1_dense (Dense)        (None, 150, 64)           4992      
_________________________________________________________________
block_1_activation (Activati (None, 150, 64)           0         
_________________________________________________________________
block_2_bidirectional (Bidir (None, 128)               66048     
_________________________________________________________________
block_3_dense (Dense)        (None, 1)                 129       
Total params: 71,169
Trainable params: 71,169
Non-trainable params: 0
_________________________________________________________________
